In [93]:
from CitesphereConnector import CitesphereConnector

In [63]:
from CitesphereConnectorTest import *

In [64]:
import math

In [247]:
auth_object = authObject()
auth_object.authType  = 'oauth'
auth_object.access_token="0d041f05-5cf3-42df-aba3-fc7090c84437"
connector = CitesphereConnector("https://diging-dev.asu.edu/citesphere-review/api", auth_object)
print(connector.get_groups())

[{'id': 2310918, 'name': 'laubichler-lab', 'version': 3, 'created': '2019-03-28T21:40:50Z', 'lastModified': '2023-09-07T20:08:57Z', 'numItems': 311, 'owner': 342862, 'type': 'Private', 'description': '', 'url': '', 'syncInfo': None}, {'id': 2256709, 'name': 'dev-test', 'version': 41, 'created': '2018-11-19T21:34:21Z', 'lastModified': '2023-10-20T17:27:04Z', 'numItems': 85, 'owner': 342862, 'type': 'Private', 'description': '<p>This is for testing citesphere.</p>', 'url': '', 'syncInfo': None}]


In [94]:
print(connector.get_group_info(2310918))

{'id': 2310918, 'name': 'laubichler-lab', 'version': 3, 'created': '2019-03-28T21:40:50Z', 'lastModified': '2023-09-07T20:08:57Z', 'numItems': 311, 'owner': 342862, 'type': 'Private', 'description': '', 'url': '', 'syncInfo': {'createdOn': '2023-10-13T17:52:08Z', 'status': 'DONE'}}


In [67]:
groups= connector.get_groups()

In [68]:
groups

[{'id': 2310918,
  'name': 'laubichler-lab',
  'version': 3,
  'created': '2019-03-28T21:40:50Z',
  'lastModified': '2023-09-07T20:08:57Z',
  'numItems': 311,
  'owner': 342862,
  'type': 'Private',
  'description': '',
  'url': '',
  'syncInfo': None},
 {'id': 2256709,
  'name': 'dev-test',
  'version': 41,
  'created': '2018-11-19T21:34:21Z',
  'lastModified': '2023-10-20T17:27:04Z',
  'numItems': 85,
  'owner': 342862,
  'type': 'Private',
  'description': '<p>This is for testing citesphere.</p>',
  'url': '',
  'syncInfo': None}]

In [69]:
group_items = connector.get_group_items(2310918)

In [189]:
len(group_items["items"])

50

In [71]:
collections = connector.get_collections(2310918)

In [72]:
for x in collections["collections"]:
    print(x)

{'id': {'timestamp': 1697219709, 'date': 1697219709000}, 'key': 'CCBBAEJJ', 'version': 65, 'contentVersion': 0, 'numberOfCollections': 0, 'numberOfItems': 6, 'name': 'Gibson, Abraham', 'parentCollectionKey': None, 'lastModified': None, 'groupId': '2310918'}
{'id': {'timestamp': 1697219709, 'date': 1697219709000}, 'key': '8N29G786', 'version': 42, 'contentVersion': 0, 'numberOfCollections': 0, 'numberOfItems': 26, 'name': 'Daniels, Bryan', 'parentCollectionKey': None, 'lastModified': None, 'groupId': '2310918'}
{'id': {'timestamp': 1697219709, 'date': 1697219709000}, 'key': '6JH7V39H', 'version': 37, 'contentVersion': 0, 'numberOfCollections': 0, 'numberOfItems': 3, 'name': 'VelezCruz, Nayely', 'parentCollectionKey': None, 'lastModified': None, 'groupId': '2310918'}
{'id': {'timestamp': 1697219709, 'date': 1697219709000}, 'key': 'AV7UJKNF', 'version': 31, 'contentVersion': 0, 'numberOfCollections': 0, 'numberOfItems': 4, 'name': 'Aiello, Kenneth', 'parentCollectionKey': None, 'lastModif

In [73]:
for x in collections["collections"]:
    x["pages"] = math.ceil(x["numberOfItems"]/50)

In [74]:
for x in collections["collections"]:
    print(x)

{'id': {'timestamp': 1697219709, 'date': 1697219709000}, 'key': 'CCBBAEJJ', 'version': 65, 'contentVersion': 0, 'numberOfCollections': 0, 'numberOfItems': 6, 'name': 'Gibson, Abraham', 'parentCollectionKey': None, 'lastModified': None, 'groupId': '2310918', 'pages': 1}
{'id': {'timestamp': 1697219709, 'date': 1697219709000}, 'key': '8N29G786', 'version': 42, 'contentVersion': 0, 'numberOfCollections': 0, 'numberOfItems': 26, 'name': 'Daniels, Bryan', 'parentCollectionKey': None, 'lastModified': None, 'groupId': '2310918', 'pages': 1}
{'id': {'timestamp': 1697219709, 'date': 1697219709000}, 'key': '6JH7V39H', 'version': 37, 'contentVersion': 0, 'numberOfCollections': 0, 'numberOfItems': 3, 'name': 'VelezCruz, Nayely', 'parentCollectionKey': None, 'lastModified': None, 'groupId': '2310918', 'pages': 1}
{'id': {'timestamp': 1697219709, 'date': 1697219709000}, 'key': 'AV7UJKNF', 'version': 31, 'contentVersion': 0, 'numberOfCollections': 0, 'numberOfItems': 4, 'name': 'Aiello, Kenneth', 'pa

In [75]:
csv_items=[]
for x in collections["collections"]:
    if x["pages"] == 1:
        itm = connector.get_collection_items_pg(2310918, x["key"],1)
        for value in itm["items"]:
            csv_items.append(value)
    else:
        for i in range(x["pages"]):
            itm = connector.get_collection_items_pg(2310918, x["key"],i+1)
            for value in itm["items"]:
                csv_items.append(value)

In [76]:
print(len(csv_items))

322


In [77]:
import csv

with open('citesphere.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    field = list(csv_items[0].keys())
    
    writer.writerow(field)
    for i in range(len(csv_items)):
        writer.writerow(list(csv_items[i].values()))

In [78]:
import json

# Assuming dev is a dictionary
dev = connector.get_collection_items_pg(2256709, "APERMCMB", 1)



In [79]:
dev

{'group': {'id': 2256709,
  'name': 'dev-test',
  'version': 41,
  'created': '2018-11-19T21:34:21Z',
  'lastModified': '2023-10-20T17:27:04Z',
  'numItems': 85,
  'owner': 342862,
  'type': 'Private',
  'description': '<p>This is for testing citesphere.</p>',
  'url': '',
  'syncInfo': {'createdOn': '2023-10-24T17:38:27.028893Z', 'status': 'DONE'}},
 'items': [{'key': '84QRMK5M',
   'group': '2256709',
   'version': 3024,
   'title': 'Spatiotemporal dynamics of a diffusive Leslie-Gower prey-predator modelwith strong Allee effect',
   'parentItem': None,
   'authors': [{'@type': 'Person',
     'name': ' Zhan-Ping Ma',
     'uri': '',
     'localAuthorityId': '',
     'firstName': ' Zhan-Ping',
     'lastName': 'Ma',
     'positionInList': 0,
     'citationKey': None,
     'affiliations': [{'@type': 'Affiliation',
       'name': ' Henan Polytech Univ, Sch Math & Informat Sci, Jiaozuo 454003, Henan, Peoples R China.',
       'uri': '',
       'localAuthorityId': None}]},
    {'@type': 'P

In [227]:
import requests

citesphere_token = "0d041f05-5cf3-42df-aba3-fc7090c84437"
giles_url = "https://diging.asu.edu/geco-giles-staging//api/v2/resources/documents/DOCZFo2sLr9ZAzK"
headers = {
    "Authorization": f"Bearer {citesphere_token}"
}

response = requests.get(giles_url, headers=headers)

if response.status_code == 200:
    print("Authentication successful")
    print(response.text)
else:
    print(f"Authentication failed with status code {response.status_code}")
    print(response.text)


Authentication successful
{
  "documentId" : "DOCZFo2sLr9ZAzK",
  "documentStatus" : "COMPLETE",
  "uploadId" : "UPAmppPghRwuqj",
  "uploadedDate" : "2023-10-04T18:30:30.128829Z",
  "access" : "PRIVATE",
  "uploadedFile" : {
    "filename" : "Einstein_Wiki.pdf",
    "id" : "FILEQL8sKmrswS7p",
    "url" : "https://diging.asu.edu/geco-giles-staging/rest/files/FILEQL8sKmrswS7p/content",
    "path" : "pdfs/OAUTHCLIENT26_jophals/UPAmppPghRwuqj/DOCZFo2sLr9ZAzK/Einstein_Wiki.pdf",
    "content-type" : "application/pdf",
    "size" : 1675019
  },
  "extractedText" : {
    "filename" : "Einstein_Wiki.pdf.txt",
    "id" : "FILEWTgdqQ3tzLCQ",
    "url" : "https://diging.asu.edu/geco-giles-staging/rest/files/FILEWTgdqQ3tzLCQ/content",
    "path" : "texts/OAUTHCLIENT26_jophals/UPAmppPghRwuqj/DOCZFo2sLr9ZAzK/Einstein_Wiki.pdf.txt",
    "content-type" : "text/plain",
    "size" : 183548
  },
  "additionalFiles" : [ {
    "filename" : "image_-1332034451.png",
    "id" : "FILEvMwkVZdVm5eS",
    "url" :

In [135]:
response.__dict__.keys()

dict_keys(['_content', '_content_consumed', '_next', 'status_code', 'headers', 'raw', 'url', 'encoding', 'history', 'reason', 'cookies', 'elapsed', 'request', 'connection'])

In [136]:
dev["items"]

[{'key': '84QRMK5M',
  'group': '2256709',
  'version': 3024,
  'title': 'Spatiotemporal dynamics of a diffusive Leslie-Gower prey-predator modelwith strong Allee effect',
  'parentItem': None,
  'authors': [{'@type': 'Person',
    'name': ' Zhan-Ping Ma',
    'uri': '',
    'localAuthorityId': '',
    'firstName': ' Zhan-Ping',
    'lastName': 'Ma',
    'positionInList': 0,
    'citationKey': None,
    'affiliations': [{'@type': 'Affiliation',
      'name': ' Henan Polytech Univ, Sch Math & Informat Sci, Jiaozuo 454003, Henan, Peoples R China.',
      'uri': '',
      'localAuthorityId': None}]},
   {'@type': 'Person',
    'name': 'Radhika Coomaraswamy',
    'uri': 'http://viaf.org/viaf/61627064/',
    'localAuthorityId': '',
    'firstName': 'Radhika',
    'lastName': 'Coomaraswamy',
    'positionInList': 0,
    'citationKey': None,
    'affiliations': []},
   {'@type': 'Person',
    'name': 'Zhan-Ping Ma',
    'uri': '',
    'localAuthorityId': '',
    'firstName': 'Zhan-Ping',
    

In [143]:
dev_group_items = connector.get_group_items(2256709)

In [148]:
dev_collections = connector.get_collections(2256709)

In [172]:
dev_collections["collections"]

[{'id': {'timestamp': 1698169144, 'date': 1698169144000},
  'key': '5FVDFAZV',
  'version': 3098,
  'contentVersion': 0,
  'numberOfCollections': 0,
  'numberOfItems': 0,
  'name': 'RSE',
  'parentCollectionKey': None,
  'lastModified': None,
  'groupId': '2256709'},
 {'id': {'timestamp': 1698169144, 'date': 1698169144000},
  'key': 'APERMCMB',
  'version': 290,
  'contentVersion': 0,
  'numberOfCollections': 0,
  'numberOfItems': 13,
  'name': 'test2',
  'parentCollectionKey': None,
  'lastModified': None,
  'groupId': '2256709'},
 {'id': {'timestamp': 1698169145, 'date': 1698169145000},
  'key': 'WAERDMDZ',
  'version': 920,
  'contentVersion': 0,
  'numberOfCollections': 0,
  'numberOfItems': 79,
  'name': 'test21',
  'parentCollectionKey': None,
  'lastModified': None,
  'groupId': '2256709'},
 {'id': {'timestamp': 1698169145, 'date': 1698169145000},
  'key': 'SU742HVM',
  'version': 919,
  'contentVersion': 0,
  'numberOfCollections': 0,
  'numberOfItems': 44,
  'name': 'test20',


In [157]:
testing = connector.get_collection_items(2256709,"TN2TAZNW")

In [158]:
testing

{'group': {'id': 2256709,
  'name': 'dev-test',
  'version': 41,
  'created': '2018-11-19T21:34:21Z',
  'lastModified': '2023-10-20T17:27:04Z',
  'numItems': 85,
  'owner': 342862,
  'type': 'Private',
  'description': '<p>This is for testing citesphere.</p>',
  'url': '',
  'syncInfo': {'createdOn': '2023-10-24T17:38:27.028893Z', 'status': 'DONE'}},
 'items': [{'key': 'FDGBE2AZ',
   'group': '2256709',
   'version': 2992,
   'title': 'Unilateral sources and sinks of an activator in reaction-diffusionsystems exhibiting diffusion-driven instability',
   'parentItem': None,
   'authors': [{'@type': 'Person',
     'name': ' Martin Fencl',
     'uri': '',
     'localAuthorityId': '',
     'firstName': ' Martin',
     'lastName': 'Fencl',
     'positionInList': 0,
     'citationKey': None,
     'affiliations': [{'@type': 'Affiliation',
       'name': ' Univ West Bohemia, Fac Appl Sci, NTIS, Univ 8, Plzen 30100, Czech Republic.',
       'uri': '',
       'localAuthorityId': None},
      {'@t

In [169]:
testing["items"]

[{'key': 'FDGBE2AZ',
  'group': '2256709',
  'version': 2992,
  'title': 'Unilateral sources and sinks of an activator in reaction-diffusionsystems exhibiting diffusion-driven instability',
  'parentItem': None,
  'authors': [{'@type': 'Person',
    'name': ' Martin Fencl',
    'uri': '',
    'localAuthorityId': '',
    'firstName': ' Martin',
    'lastName': 'Fencl',
    'positionInList': 0,
    'citationKey': None,
    'affiliations': [{'@type': 'Affiliation',
      'name': ' Univ West Bohemia, Fac Appl Sci, NTIS, Univ 8, Plzen 30100, Czech Republic.',
      'uri': '',
      'localAuthorityId': None},
     {'@type': 'Affiliation',
      'name': ' Univ West Bohemia, Fac Appl Sci, Dept Math, Univ 8, Plzen 30100, Czech Republic.',
      'uri': '',
      'localAuthorityId': None}]},
   {'@type': 'Person',
    'name': ' Milan Kucera',
    'uri': '',
    'localAuthorityId': '',
    'firstName': ' Milan',
    'lastName': 'Kucera',
    'positionInList': 0,
    'citationKey': None,
    'affil

In [181]:
get_file_ids(2256709,"TN2TAZNW")

FILESSuu8mSuZRdM


In [182]:
groups = connector.get_groups()

In [185]:
group_ids = [x["id"] for x in groups]

In [222]:
def get_file_ids(zotero_id,collection_id):
    testing = connector.get_collection_items(zotero_id,collection_id)
    file_ids=[]
    for x in testing["items"]:
        if type(x["gilesUploads"]) == type([1,2,3]) and len(x["gilesUploads"])>0:
            for y in x["gilesUploads"]:
                 file_ids.append(y["uploadedFile"]["id"]) 

In [204]:
def get_all_collections(zotero_id):
    collections_ = connector.get_collections(zotero_id)
    collections_keys = [c["key"] for c in collections_["collections"]]
    return {zotero_id: collections_keys}

In [205]:
def Merge(dict1, dict2):
    return(dict2.update(dict1))

collections_dict={}
for x in group_ids:
    Merge(get_all_collections(x),collections_dict)
collections_dict

{2310918: ['CCBBAEJJ',
  '8N29G786',
  '6JH7V39H',
  'AV7UJKNF',
  'SFZ7JWTD',
  'SQ3CWKC4',
  'A73U8NXR'],
 2256709: ['5FVDFAZV',
  'APERMCMB',
  'WAERDMDZ',
  'SU742HVM',
  'Q6ZQKU7K',
  'D6BPEP8S',
  'U3XJET94',
  'DNUJIMND',
  'CHEQX2JU',
  'HU963G6Z',
  'ZVAQ6ITQ',
  'HDDSEIDQ',
  'DVQDT3M9',
  '7I2Q8M8F',
  '47QPK27S',
  '2IX9EI22',
  'MQTVXH8H',
  'TN2TAZNW',
  'S9UZ98XH',
  'D25HU8SW',
  '4NVRZWB3',
  '2RQTF8C2']}

In [223]:
files_list=[]
for x in collections_dict:
    for y in collections_dict[x]:
        files_list.append(get_file_ids(x,y))

FILEQL8sKmrswS7p
FILEfItViY7iUgLM
FILEQL8sKmrswS7p
FILE3u01O8ugYrge
FILESSuu8mSuZRdM
FILEwUpCCo2Y5Tmx
FILEwUpCCo2Y5Tmx
FILEcHWoIjAxJSlV
FILEwUpCCo2Y5Tmx
FILEcHWoIjAxJSlV
FILEcHWoIjAxJSlV
FILENf3i3D2v2XkG
FILEwUpCCo2Y5Tmx
FILEcHWoIjAxJSlV
FILEwUpCCo2Y5Tmx
FILESSuu8mSuZRdM
FILEwUpCCo2Y5Tmx
FILEcHWoIjAxJSlV
FILEcpf2PgsbX0YR
FILEcHWoIjAxJSlV
FILESSuu8mSuZRdM
FILEwUpCCo2Y5Tmx
FILEcHWoIjAxJSlV
FILEwUpCCo2Y5Tmx
FILEcHWoIjAxJSlV
FILEcHWoIjAxJSlV
FILESSuu8mSuZRdM
FILEfItViY7iUgLM
FILEwUpCCo2Y5Tmx
FILEcHWoIjAxJSlV
FILEwUpCCo2Y5Tmx
FILEcHWoIjAxJSlV
FILEwUpCCo2Y5Tmx
FILEcHWoIjAxJSlV


In [229]:
lab = connector.get_collection_items_pg(2310918, "A73U8NXR", 1)

In [230]:
lab["items"]

[{'key': '8GCTQ3XH',
  'group': '2310918',
  'version': 1604,
  'title': '\u202aCOVID-19 heralds a new epistemology of science for the public good\u202c',
  'parentItem': None,
  'authors': [{'@type': 'Person',
    'name': ' LAUBICHLER et. al.',
    'uri': None,
    'localAuthorityId': None,
    'firstName': '',
    'lastName': 'LAUBICHLER et. al.',
    'positionInList': 0,
    'citationKey': None,
    'affiliations': None},
   {'@type': 'Person',
    'name': 'P Schlosser',
    'uri': None,
    'localAuthorityId': None,
    'firstName': 'P',
    'lastName': 'Schlosser',
    'positionInList': 1,
    'citationKey': None,
    'affiliations': None},
   {'@type': 'Person',
    'name': 'J Renn',
    'uri': None,
    'localAuthorityId': None,
    'firstName': 'J',
    'lastName': 'Renn',
    'positionInList': 2,
    'citationKey': None,
    'affiliations': None}],
  'editors': [],
  'otherCreators': [],
  'itemType': 'JOURNAL_ARTICLE',
  'publicationTitle': '',
  'volume': '43',
  'issue': '2

In [241]:
dev_dict=connector.get_collections(2256709)

In [246]:
for z in dev_dict["collections"]:
    print(z["numberOfItems"])

0
13
79
44
26
27
20
26
15
23
24
24
29
21
25
23
16
13
38
17
27
18
